In [ ]:
import pandas as pd
import numpy as np
import random
import math 
import sys

Algorytm Tabu Search oraz Simulated Annelaling wraz z funkcjami potrzebnymi do ich wywołania

In [ ]:
def df_to_array(excel_data):
  data = pd.read_excel(excel_data)
  data.pop('Unnamed: 0')
  citiesLength = data
  citiesLength = np.array(citiesLength)
  cityNumbers = list(range(0, len(data)))
  return citiesLength, cityNumbers

def generateRandomSolution(indices):
  indices = indices.copy()
  randomSolution = []
  for i in range(0, len(indices)):
    randomIndex = random.randint(0, len(indices) - 1)
    randomSolution.append(indices[randomIndex])
    indices.pop(randomIndex)
  return randomSolution

def distanceBetweenTwoPoints(data, pointOne, pointTwo):
  return data[pointOne][pointTwo]

def allSolutionDistance(data, indices):
  sum = 0
  for i in range(len(indices) - 1):
    sum += distanceBetweenTwoPoints(data, indices[i], indices[i+1])
  return (sum + distanceBetweenTwoPoints(data, indices[len(indices) - 1], indices[0]))

def createCombinations(indices):
  combinations = []
  for i in range(0, len(indices)):
    for j in range(i+1, (len(indices))):
      combinations.append([indices[i], indices[j]])
  return combinations

def divisionNeighbour(array, low, high):
  pivot = array[high][1]
  i = low - 1
  for j in range(low, high):
      if array[j][1] <= pivot:
          i = i + 1
          (array[i][1], array[j][1]) = (array[j][1], array[i][1])
          (array[i][0], array[j][0]) = (array[j][0], array[i][0])
          (array[i][2], array[j][2]) = (array[j][2], array[i][2])
  (array[i + 1][1], array[high][1]) = (array[high][1], array[i + 1][1])
  (array[i + 1][0], array[high][0]) = (array[high][0], array[i + 1][0])
  (array[i + 1][2], array[high][2]) = (array[high][2], array[i + 1][2])
  return i + 1
  
def quickSortNeighboursByF(array, low, high):
  if low < high:
      pi = divisionNeighbour(array, low, high)
      quickSortNeighboursByF(array, low, pi - 1)
      quickSortNeighboursByF(array, pi + 1, high)

def createNeighbourhoodsSwap(parent, combinations, data):
  neighbours = []
  for i in range(len(combinations)):
    solution = []
    for j in range(len(parent)):
      if parent[j] == combinations[i][0]:
        solution.append(combinations[i][1])
      elif parent[j] == combinations[i][1]:
        solution.append(combinations[i][0])
      else:
        solution.append(parent[j])
    neighbours.append([solution, allSolutionDistance(data, solution), combinations[i]])
  quickSortNeighboursByF(neighbours, 0, len(neighbours) - 1)
  return neighbours

def createNeighbourhoodsInvert(parent, combinations, data):
  neighbours = []
  for k in range(len(combinations)):
    solution = []
    index1 = combinations[k][0]
    index2 = combinations[k][1]
    Xlen = len(parent)
    if index1 > index2:
      helper = index1
      index1 = index2
      index2 = helper
    i = 0
    j = 0
    while i < index1:
      solution.append(parent[i])
      i = i + 1
    i = index2
    while i >= index1:
      solution.append(parent[i])
      i = i - 1
    i = index2 + 1
    while i <= Xlen - 1:
      solution.append(parent[i])
      i = i + 1
    neighbours.append([solution, allSolutionDistance(data, solution), combinations[k]])
  quickSortNeighboursByF(neighbours, 0, len(neighbours) - 1)
  return neighbours

def changeIndices(indices):
  return list(map(lambda x: x+1, indices))
  
def createSolutionInvert(X):
  arr = []
  Xlen = len(X)
  index1 = random.randint(0, Xlen - 1)
  index2 = random.randint(0, Xlen - 1)
  while index1 == index2:
    index2 = random.randint(0, Xlen - 1)
  if index1 > index2:
    helper = index1
    index1 = index2
    index2 = helper
  i = 0
  j = 0
  while i < index1:
    arr.append(X[i])
    i = i + 1
  i = index2
  while i >= index1:
    arr.append(X[i])
    i = i - 1
  i = index2 + 1
  while i <= Xlen - 1:
    arr.append(X[i])
    i = i + 1
  return arr

def createSolutionSwap(X):
  arr = []
  Xlen = len(X)
  index1 = random.randint(0, Xlen - 1)
  index2 = random.randint(0, Xlen - 1)
  for j in range(Xlen):
      if X[j] == index1:
        arr.append(index2)
      elif X[j] == index2:
        arr.append(index1)
      else:
        arr.append(X[j])
  return arr

def changeIndicesInCombination(combination):
  combination[0], combination[1] = combination[1], combination[0]
  return combination

def TabuSearch(data_name, createNeighbourhoodOfType, term, iterations):
  citiesLength, cityNumbers = df_to_array(data_name)
  X0 = generateRandomSolution(cityNumbers)
  F0 = allSolutionDistance(citiesLength, X0)
  Xbest = X0
  Fbest = F0
  Xcurrent = X0
  Fcurrent = F0
  tabuList = []
  for i in range(iterations):
    combinations = createCombinations(Xcurrent)
    neighbours = createNeighbourhoodOfType(Xcurrent, combinations, citiesLength)
    neighbour = neighbours[0]
    Xcurrent = neighbour[0]
    Fcurrent = neighbour[1]
    currentComb = neighbour[2]
    if currentComb[0] > currentComb[1]:
      changeIndices(currentComb)
    j = 0
    while currentComb in tabuList:
      j = j + 1
      currentComb = neighbours[j][2]
      if currentComb[0] > currentComb[1]:
        changeIndices(currentComb)
    neighbour = neighbours[j]
    Xcurrent = neighbour[0]
    Fcurrent = neighbour[1]
    if Fcurrent < Fbest:
      Fbest = Fcurrent
      Xbest = Xcurrent
    tabuList.append(currentComb)
    if(i > term): 
      tabuList.pop(0)
  Xbest = changeIndices(Xbest)
  return Fbest, Xbest

def SimulatedAnnealing(data_name, createNeighbourhoodOfType, initialT, alpha, iterations):
  citiesLength, cityNumbers = df_to_array(data_name)
  X0 = generateRandomSolution(cityNumbers)
  F0 = allSolutionDistance(citiesLength, X0)
  Xbest = X0
  Fbest = F0
  Xcurrent = X0
  Fcurrent = F0
  currentT = float(initialT)
  for i in range(iterations):
    Xpropose = createSolutionInvert(Xcurrent)
    Fpropose = allSolutionDistance(citiesLength, Xpropose)
    error =  float(Fcurrent - Fpropose)
    if error >= 0:
      Fcurrent = Fpropose
      Xcurrent = Xpropose
    else:
      pAccept = math.exp(error/currentT)
      if random.uniform(0, 1) < pAccept:
        Fcurrent = Fpropose
        Xcurrent = Xpropose
    if Fcurrent < Fbest:
      Fbest = Fcurrent
      Xbest = Xcurrent
    currentT = currentT * alpha
  Xbest = changeIndices(Xbest)
  return Fbest, Xbest

Przykładowe wywołania algorytmów dla różnych parametrów

In [ ]:
Fbest = sys.maxsize
Xbest = []
n = 30
for i in range(n):
  Fcurrent, Xcurrent = TabuSearch('Dane_TSP_127.xlsx', createNeighbourhoodsInvert, 70, 300)
  if Fbest > Fcurrent:
    Fbest = Fcurrent
    Xbest = Xcurrent
print(Fbest, Xbest)

In [ ]:
TabuSearch('Dane_TSP_48.xlsx', createNeighbourhoodsInvert, 10, 300)

In [ ]:
Fbest = sys.maxsize
Xbest = []
n = 5
for i in range(n):
  Fcurrent, Xcurrent = SimulatedAnnealing('Dane_TSP_48.xlsx', createNeighbourhoodsInvert, 3900, 0.94, 2000)
  if Fbest > Fcurrent:
    Fbest = Fcurrent
    Xbest = Xcurrent
print(Fbest, Xbest)

In [ ]:
terms = [2,10,30,70,150]
iterations = [30, 100, 150, 300, 900, 1800]
Fbest = sys.maxsize
meanBest = sys.maxsize
Xbest = []
iterationBest = 0
termBest = 0
n = 5
for iteration in iterations:
  for term in terms:
    fSum = 0
    for i in range(n):
      Fcurrent, Xcurrent = TabuSearch('Dane_TSP_48.xlsx', createNeighbourhoodsInvert, term, iteration)
      if Fbest > Fcurrent:
        Fbest = Fcurrent
        Xbest = Xcurrent
        iterationBest = iteration
        termBest = term
      fSum = fSum + Fcurrent
    fmean = fSum/n
    if fmean < meanBest:
      meanBest = fmean
      meanIter = iteration
      meanTerm = term
    print('Mean:', fSum/n, 'iterations:', iteration, 'term', term)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, termBest)
print('Najlepsza średnia:', meanBest, 'iterations:', meanIter, 'term', meanTerm)


In [ ]:
terms = [2,10,30,70,150]
iterations = [30, 100, 150, 300, 900, 1800]
Fbest = sys.maxsize
meanBest = sys.maxsize
Xbest = []
iterationBest = 0
termBest = 0
n = 5
for iteration in iterations:
  for term in terms:
    fSum = 0
    for i in range(n):
      Fcurrent, Xcurrent = TabuSearch('Dane_TSP_48.xlsx', createNeighbourhoodsSwap, term, iteration)
      if Fbest > Fcurrent:
        Fbest = Fcurrent
        Xbest = Xcurrent
        iterationBest = iteration
        termBest = term
      fSum = fSum + Fcurrent
    fmean = fSum/n
    if fmean < meanBest:
      meanBest = fmean
      meanIter = iteration
      meanTerm = term
    print('Mean:', fSum/n, 'iterations:', iteration, 'term', term)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, termBest)
print('Najlepsza średnia:', meanBest, 'iterations:', meanIter, 'term', meanTerm)


In [ ]:
terms = [2,10,30,70,150]
iterations = [30, 100, 150, 300, 900, 1800]
Fbest = sys.maxsize
meanBest = sys.maxsize
Xbest = []
iterationBest = 0
termBest = 0
n = 5
for iteration in iterations:
  for term in terms:
    fSum = 0
    for i in range(n):
      Fcurrent, Xcurrent = TabuSearch('Dane_TSP_76.xlsx', createNeighbourhoodsInvert, term, iteration)
      if Fbest > Fcurrent:
        Fbest = Fcurrent
        Xbest = Xcurrent
        iterationBest = iteration
        termBest = term
      fSum = fSum + Fcurrent
    fmean = fSum/n
    if fmean < meanBest:
      meanBest = fmean
      meanIter = iteration
      meanTerm = term
    print('Mean:', fSum/n, 'iterations:', iteration, 'term', term)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, termBest)
print('Najlepsza średnia:', meanBest, 'iterations:', meanIter, 'term', meanTerm)


In [ ]:
terms = [2,10,30,70,150]
iterations = [30, 100, 150, 300, 900, 1800, 2400]
Fbest = sys.maxsize
meanBest = sys.maxsize
Xbest = []
iterationBest = 0
termBest = 0
n = 5
for iteration in iterations:
  for term in terms:
    fSum = 0
    for i in range(n):
      Fcurrent, Xcurrent = TabuSearch('Dane_TSP_76.xlsx', createNeighbourhoodsSwap, term, iteration)
      if Fbest > Fcurrent:
        Fbest = Fcurrent
        Xbest = Xcurrent
        iterationBest = iteration
        termBest = term
      fSum = fSum + Fcurrent
    fmean = fSum/n
    if fmean < meanBest:
      meanBest = fmean
      meanIter = iteration
      meanTerm = term
    print('Mean:', fSum/n, 'iterations:', iteration, 'term', term)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, termBest)
print('Najlepsza średnia:', meanBest, 'iterations:', meanIter, 'term', meanTerm)


In [ ]:
terms = [2,10,30,70,150]
iterations = [30, 100, 150, 300, 900, 1800, 2400]
Fbest = sys.maxsize
meanBest = sys.maxsize
Xbest = []
iterationBest = 0
termBest = 0
n = 5
for iteration in iterations:
  for term in terms:
    fSum = 0
    for i in range(n):
      Fcurrent, Xcurrent = TabuSearch('Dane_TSP_127.xlsx', createNeighbourhoodsInvert, term, iteration)
      if Fbest > Fcurrent:
        Fbest = Fcurrent
        Xbest = Xcurrent
        iterationBest = iteration
        termBest = term
      fSum = fSum + Fcurrent
    fmean = fSum/n
    if fmean < meanBest:
      meanBest = fmean
      meanIter = iteration
      meanTerm = term
    print('Mean:', fSum/n, 'iterations:', iteration, 'term', term)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, termBest)
print('Najlepsza średnia:', meanBest, 'iterations:', meanIter, 'term', meanTerm)

In [ ]:
terms = [2,10,30,70,150]
iterations = [30, 100, 150, 300, 900, 1800, 2400]
Fbest = sys.maxsize
meanBest = sys.maxsize
Xbest = []
iterationBest = 0
termBest = 0
n = 5
for iteration in iterations:
  for term in terms:
    fSum = 0
    for i in range(n):
      Fcurrent, Xcurrent = TabuSearch('Dane_TSP_127.xlsx', createNeighbourhoodsSwap, term, iteration)
      if Fbest > Fcurrent:
        Fbest = Fcurrent
        Xbest = Xcurrent
        iterationBest = iteration
        termBest = term
      fSum = fSum + Fcurrent
    fmean = fSum/n
    if fmean < meanBest:
      meanBest = fmean
      meanIter = iteration
      meanTerm = term
    print('Mean:', fSum/n, 'iterations:', iteration, 'term', term)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, termBest)
print('Najlepsza średnia:', meanBest, 'iterations:', meanIter, 'term', meanTerm)

In [ ]:
initialTemperatures = [5000, 8000, 14000, 20000, 40000, 70000]
alphas = [0.98, 0.94, 0.89, 0.85, 0.8]
iterations = [1000, 2000, 5000, 10000, 15000]
Fbest = sys.maxsize
Xbest = []
iterationBest = 0
tempBest = 0
alphaBest = 0
n = 5
meanBest = sys.maxsize
for iteration in iterations:
  for temp in initialTemperatures:
    for alpha in alphas:
      fSum = 0
      for i in range(n):
        Fcurrent, Xcurrent = SimulatedAnnealing('Dane_TSP_48.xlsx', createSolutionSwap, temp, alpha, iteration)
        if Fbest > Fcurrent:
          Fbest = Fcurrent
          Xbest = Xcurrent
          iterationBest = iteration
          tempBest = temp
          alphaBest = alpha
        fSum = fSum + Fcurrent
      fmean = fSum/n
      if fmean < meanBest:
        meanBest = fmean
        meanIter = iteration
        meanTemp = temp
        meanAlpha = alpha
      print('Mean:', fSum/n, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, tempBest, alphaBest)
print('najlepsza średnia:', meanBest, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)

In [ ]:
initialTemperatures = [5000, 8000, 20000, 40000, 70000]
alphas = [0.98, 0.94, 0.89, 0.85, 0.8]
iterations = [1000, 2000, 5000, 10000, 15000]
Fbest = sys.maxsize
Xbest = []
iterationBest = 0
tempBest = 0
alphaBest = 0
n = 5
meanBest = sys.maxsize
for iteration in iterations:
  for temp in initialTemperatures:
    for alpha in alphas:
      fSum = 0
      for i in range(n):
        Fcurrent, Xcurrent = SimulatedAnnealing('Dane_TSP_48.xlsx', createSolutionInvert, temp, alpha, iteration)
        if Fbest > Fcurrent:
          Fbest = Fcurrent
          Xbest = Xcurrent
          iterationBest = iteration
          tempBest = temp
          alphaBest = alpha
        fSum = fSum + Fcurrent
      fmean = fSum/n
      if fmean < meanBest:
        meanBest = fmean
        meanIter = iteration
        meanTemp = temp
        meanAlpha = alpha
      print('Mean:', fSum/n, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, tempBest, alphaBest)
print('najlepsza średnia:', meanBest, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)

In [ ]:
initialTemperatures = [5000, 8000, 14000, 20000, 40000, 70000]
alphas = [0.98, 0.94, 0.89, 0.85, 0.8]
iterations = [1000, 2000, 5000, 10000, 15000]
Fbest = sys.maxsize
Xbest = []
iterationBest = 0
tempBest = 0
alphaBest = 0
n = 5
meanBest = sys.maxsize
for iteration in iterations:
  for temp in initialTemperatures:
    for alpha in alphas:
      fSum = 0
      for i in range(n):
        Fcurrent, Xcurrent = SimulatedAnnealing('Dane_TSP_76.xlsx', createSolutionSwap, temp, alpha, iteration)
        if Fbest > Fcurrent:
          Fbest = Fcurrent
          Xbest = Xcurrent
          iterationBest = iteration
          tempBest = temp
          alphaBest = alpha
        fSum = fSum + Fcurrent
      fmean = fSum/n
      if fmean < meanBest:
        meanBest = fmean
        meanIter = iteration
        meanTemp = temp
        meanAlpha = alpha
      print('Mean:', fSum/n, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, tempBest, alphaBest)
print('najlepsza średnia:', meanBest, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)

In [ ]:
initialTemperatures = [5000, 14000, 20000, 40000, 70000]
alphas = [0.98, 0.94, 0.89, 0.85, 0.8]
iterations = [1000, 2000, 5000, 10000, 15000]
Fbest = sys.maxsize
Xbest = []
iterationBest = 0
tempBest = 0
alphaBest = 0
n = 5
meanBest = sys.maxsize
for iteration in iterations:
  for temp in initialTemperatures:
    for alpha in alphas:
      fSum = 0
      for i in range(n):
        Fcurrent, Xcurrent = SimulatedAnnealing('Dane_TSP_76.xlsx', createSolutionInvert, temp, alpha, iteration)
        if Fbest > Fcurrent:
          Fbest = Fcurrent
          Xbest = Xcurrent
          iterationBest = iteration
          tempBest = temp
          alphaBest = alpha
        fSum = fSum + Fcurrent
      fmean = fSum/n
      if fmean < meanBest:
        meanBest = fmean
        meanIter = iteration
        meanTemp = temp
        meanAlpha = alpha
      print('Mean:', fSum/n, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, tempBest, alphaBest)
print('najlepsza średnia:', meanBest, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)

In [ ]:
initialTemperatures = [5000, 8000, 14000, 20000, 40000, 70000, 100000]
alphas = [0.98, 0.94, 0.89, 0.85, 0.8]
iterations = [1000, 2000, 5000, 10000, 15000]
Fbest = sys.maxsize
Xbest = []
iterationBest = 0
tempBest = 0
alphaBest = 0
n = 5
meanBest = sys.maxsize
for iteration in iterations:
  for temp in initialTemperatures:
    for alpha in alphas:
      fSum = 0
      for i in range(n):
        Fcurrent, Xcurrent = SimulatedAnnealing('Dane_TSP_127.xlsx', createSolutionSwap, temp, alpha, iteration)
        if Fbest > Fcurrent:
          Fbest = Fcurrent
          Xbest = Xcurrent
          iterationBest = iteration
          tempBest = temp
          alphaBest = alpha
        fSum = fSum + Fcurrent
      fmean = fSum/n
      if fmean < meanBest:
        meanBest = fmean
        meanIter = iteration
        meanTemp = temp
        meanAlpha = alpha
      print('Mean:', fSum/n, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, tempBest, alphaBest)
print('najlepsza średnia:', meanBest, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)

In [ ]:
initialTemperatures = [8000, 14000, 20000, 40000, 70000, 100000]
alphas = [0.98, 0.94, 0.89, 0.85, 0.8]
iterations = [1000, 2000, 5000, 10000, 15000]
Fbest = sys.maxsize
Xbest = []
iterationBest = 0
tempBest = 0
alphaBest = 0
n = 5
meanBest = sys.maxsize
for iteration in iterations:
  for temp in initialTemperatures:
    for alpha in alphas:
      fSum = 0
      for i in range(n):
        Fcurrent, Xcurrent = SimulatedAnnealing('Dane_TSP_127.xlsx', createSolutionInvert, temp, alpha, iteration)
        if Fbest > Fcurrent:
          Fbest = Fcurrent
          Xbest = Xcurrent
          iterationBest = iteration
          tempBest = temp
          alphaBest = alpha
        fSum = fSum + Fcurrent
      fmean = fSum/n
      if fmean < meanBest:
        meanBest = fmean
        meanIter = iteration
        meanTemp = temp
        meanAlpha = alpha
      print('Mean:', fSum/n, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)
print('Najlepszy wynik:', Fbest, Xbest, iterationBest, tempBest, alphaBest)
print('najlepsza średnia:', meanBest, 'iterations:', iteration, 'temp', temp, 'alpha', alpha)